*source: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews*

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [25]:
df = pd.read_csv("../data/books_data.csv")
# df_ratings = pd.read_csv("../data/Books_rating.csv")

In [18]:
df.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [19]:
print(df.shape)
df.columns = df.columns.str.lower()
print(list(df.columns))
df.info()

(212404, 10)
['title', 'description', 'authors', 'image', 'previewlink', 'publisher', 'publisheddate', 'infolink', 'categories', 'ratingscount']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewlink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publisheddate  187099 non-null  object 
 7   infolink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingscount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [20]:
df.isna().sum()

title                 1
description       68442
authors           31413
image             52075
previewlink       23836
publisher         75886
publisheddate     25305
infolink          23836
categories        41199
ratingscount     162652
dtype: int64

### `part 1`

In [91]:
# Unique categories / genres
import ast
import pandas as pd

def parse_categories(val):
    if pd.isna(val):
        return []
    val = val.strip()

    # Case 1: Python-like list stored as string
    if val.startswith("[") and val.endswith("]"):
        try:
            parsed = ast.literal_eval(val)
            return [c.strip() for c in parsed if isinstance(c, str)]
        except Exception:
            return []
    
    # Case 2: Comma-separated string
    return [c.strip() for c in val.split(",") if c.strip()]

def normalize_genre(g):
    g = g.lower().strip()
    g = g.replace("&", "and")
    return g

df["genres"] = df["categories"].apply(parse_categories)
df["genres"] = df["genres"].apply(lambda genres: [normalize_genre(g) for g in genres])
print(df["genres"].head())

0      [comics and graphic novels]
1    [biography and autobiography]
2                       [religion]
3                        [fiction]
4                               []
Name: genres, dtype: object


In [92]:
# Unique authors
import ast
import pandas as pd

def parse_authors(val):
    if pd.isna(val):
        return []
    val = val.strip()

    # Case 1: Python-like list stored as string
    if val.startswith("[") and val.endswith("]"):
        try:
            parsed = ast.literal_eval(val)
            return [c.strip() for c in parsed if isinstance(c, str)]
        except Exception:
            return []
    
    # Case 2: Comma-separated string
    return [c.strip() for c in val.split(",") if c.strip()]

def normalize_authors(g):
    g = g.lower().strip()
    return g

df["authors"] = df["authors"].apply(parse_authors)
df["authors"] = df["authors"].apply(lambda authors: [normalize_authors(g) for g in authors])
print(df["authors"].head())

0       [julie strain]
1         [philip nel]
2       [david r. ray]
3    [veronica haddon]
4        [edward long]
Name: authors, dtype: object


In [ ]:
# How many books actually have categories now?
print(df["categories_list"].apply(len).describe())

# And how many are still empty?
print((df["categories_list"].apply(len) == 0).sum())

In [65]:
# Unique genres
all_genres = (df["categories_list"].explode().dropna().str.strip())
unique_genres = all_genres.nunique()
print(f"unique_genres: {unique_genres}")

# Most common genres
genre_counts = all_genres.value_counts()
genre_counts[genre_counts > 1].head(30)

unique_genres: 10883


categories_list
Fiction                        23419
Religion                        9459
History                         9330
Juvenile Fiction                6643
Biography & Autobiography       6324
Business & Economics            5625
Computers                       4312
Social Science                  3834
Juvenile Nonfiction             3446
Science                         2623
Education                       2611
Cooking                         2445
Sports & Recreation             2267
Family & Relationships          2178
Literary Criticism              2147
Music                           2106
Medical                         2079
Art                             2054
Body, Mind & Spirit             2049
Language Arts & Disciplines     2036
Health & Fitness                2030
Political Science               1955
Psychology                      1913
Philosophy                      1864
Travel                          1812
Technology & Engineering        1662
Self-Help             

In [66]:
genre_counts.describe()
(genre_counts == 1).sum()
genre_counts[genre_counts == 1]

categories_list
Cartas inglesas                       1
Picaresque literature                 1
Traffic safety                        1
Amber (Imaginary place)               1
Chishtīyah members                    1
                                     ..
Authors, Japanese                     1
Birth certificates                    1
Restaurant management                 1
Oxford (England)                      1
Flame in the wind (Motion picture)    1
Name: count, Length: 6535, dtype: int64

In [74]:
def normalize_genre(g):
    g = g.lower().strip()
    g = g.replace("&", "and")
    return g

df["categories_norm"] = df["categories_list"].apply(lambda lst: [normalize_genre(g) for g in lst])

all_genres = (df["categories_norm"].explode().dropna().str.strip())
unique_genres = all_genres.nunique()
print(f"unique_genres: {unique_genres}")

df["categories_norm"].explode().value_counts().head(30)

unique_genres: 10805


categories_norm
fiction                          23470
religion                          9471
history                           9347
juvenile fiction                  6671
biography and autobiography       6341
business and economics            5629
computers                         4317
social science                    3839
juvenile nonfiction               3464
science                           2629
education                         2615
cooking                           2452
sports and recreation             2268
family and relationships          2204
literary criticism                2149
music                             2109
medical                           2087
art                               2061
body, mind and spirit             2049
language arts and disciplines     2040
health and fitness                2037
political science                 1994
psychology                        1915
philosophy                        1868
travel                            1816
technolog

In [ ]:
def collapse_hierarchy(genre):
    if "/" in genre:
        parts = [p.strip() for p in genre.split("/")]
        return parts[-1].lower()  # keep last level (or first level for broad)
    return genre.lower()

df["categories_collapsed"] = df["categories_norm"].apply(lambda lst: [collapse_hierarchy(g) for g in lst])
df["categories_collapsed"].head()

all_genres = (df["categories_collapsed"].explode().dropna().str.strip())
unique_genres = all_genres.nunique()
print(f"unique_genres: {unique_genres}")

unique_genres: 10804


In [ ]:
# Fully duplicate rows
full_duplicates = df.duplicated().sum()
print(full_duplicates)

# partial duplicates
partial_duplicates = df.duplicated(subset=["Title", "authors"]).sum()
print(partial_duplicates)

In [32]:
# Descriptions containing HTML tags
html_tag_descriptions = df[df["description"].str.contains(r"<[^>]+>", na=False)].shape[0]
print(f"html_tag_descriptions: {html_tag_descriptions}")

# Descriptions with escaped characters (\t, \r, \n)
escaped_char_descriptions = df[df["description"].str.contains(r"\\[trn]", na=False)].shape[0]
print(f"escaped_char_descriptions: {escaped_char_descriptions}")

# Descriptions with HTML-escaped entities (&amp;, &#39;, etc.)
html_escaped_descriptions = df[df["description"].str.contains(r"&[#a-zA-Z0-9]+;", na=False)].shape[0]
print(f"html_escaped_descriptions: {html_escaped_descriptions}")

# Descriptions with control / non-printable characters
control_char_descriptions = df[df["description"].str.contains(r"[\x00-\x1F\x7F]", na=False)].shape[0]
print(f"control_char_descriptions: {control_char_descriptions}")

html_tag_descriptions: 17
escaped_char_descriptions: 6
html_escaped_descriptions: 63
control_char_descriptions: 2


In [38]:
# Very short descriptions
short_descriptions = df[df["description"].str.len() < 50].shape[0]
print(f"short_descriptions: {short_descriptions}")

# Extremely long descriptions (potential junk)
long_descriptions = df[df["description"].str.len() > 5000].shape[0]
print(f"long_descriptions: {long_descriptions}")

# Author count per book
author_counts = df["authors"].str.split(",").str.len()
author_counts[author_counts > 1]

short_descriptions: 2065
long_descriptions: 54


10        2.0
13        2.0
21        3.0
36        3.0
41        3.0
         ... 
212375    2.0
212379    2.0
212389    2.0
212398    2.0
212403    2.0
Name: authors, Length: 36558, dtype: float64

### `part 2`

In [26]:
INPUT_COLS = ["title", "description", "authors", "infolink", "categories"]
OUTPUT_COLS = ["title", "authors", "description", "genres", "infolink"]
COMMON_DELIMS = [";", "|", "/", "•"]
TOP_N_GENRES = 50


# EMBEDDING_MODEL = "all-mpnet-base-v2"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

In [ ]:
import ast
import html
import pandas as pd

def parse_list_value(val):
    if pd.isna(val) or val.strip() == "":
        return []
    val = val.strip()
    
    # Try to parse as Python list
    if val.startswith("[") and val.endswith("]"):
        try:
            parsed = ast.literal_eval(val)
            return [c.strip() for c in parsed if isinstance(c, str)]
        except Exception:
            return []

    # Otherwise, comma-separated
    for sep in COMMON_DELIMS:
        val = val.replace(sep, ",")
    
    return [c.strip() for c in val.split(",") if c.strip()]

def normalize_author_column(author_col):
    # basic cleanup before parsing
    cleaned_col = (author_col.fillna("").astype(str)
                    .str.replace(";", ",", regex=False)
                    .str.replace("&", ",", regex=False)
                    .str.replace(r"\s+and\s+", ",", regex=True)
                    .str.replace(r"\(.*?editor.*?\)", "", regex=True, case=False)
                    .str.replace(r"\beditor\b", "", regex=True, case=False)
                    .str.replace(r"\bed\.\b", "", regex=True, case=False)
                    .str.replace(r"\s+", " ", regex=True)
                    .str.strip())

    # parse each row value in the column
    cleaned_col = cleaned_col.apply(parse_list_value)

    # convert valid authors to title case and remove invalid ones
    invalid_authors = {"unknown"}
    cleaned_col = cleaned_col.apply(lambda author_row: sorted({item.title() for item in author_row if item.lower() not in invalid_authors}))

    return cleaned_col

def normalize_genre_column(genre_col):
    cleaned_col = genre_col.fillna("").astype(str).str.strip().str.replace("&", "and", regex=False)

    # parse each row value in the column
    cleaned_col = cleaned_col.apply(parse_list_value)
    
    # lowercase, trim, remove very short/invalid entries
    cleaned_col = cleaned_col.apply(lambda lst: [g.lower().strip() for g in lst if len(g) > 2 and "..." not in g])

    # sort   
    cleaned_col = cleaned_col.apply(lambda lst: sorted(set(lst)))
    
    return cleaned_col

def map_genres_to_top_or_other(genres_row, top_genres):
    # any genre not in top_genres → "other"
    genre_list = set()
    for g in genres_row:
        if g in top_genres:
            genre_list.add(g)
        else:
            genre_list.add("other")

    return sorted(genre_list)

def reduce_to_top_genres(genre_col):
    all_genres = genre_col.explode()
    top_genres = set(all_genres.value_counts().head(TOP_N_GENRES).index)
    return genre_col.apply(map_genres_to_top_or_other, args=(top_genres,))

def normalize_text_field(text_col):
    # Ensure string type
    text_col = text_col.fillna("").astype(str)

    # Remove HTML tags
    text_col = text_col.str.replace(r"<[^>]+>", "", regex=True)

    # Decode HTML entities
    text_col = text_col.apply(html.unescape)
    
    # Remove escaped characters
    text_col = text_col.str.replace(r"[\n\t\r]", " ", regex=True)
    
    # Collapse multiple spaces
    text_col = text_col.str.strip().str.replace(r"\s+", " ", regex=True)
    
    # Remove control/non-printable characters
    text_col = text_col.apply(lambda s: "".join(ch for ch in s if ch.isprintable()))

    return text_col


In [28]:
# Ensure expected columns exist
df.columns = df.columns.str.lower()
missing_cols = [c for c in INPUT_COLS if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing columns in input CSV: {missing_cols}")

# Only use the needed columns for processing
df = df[INPUT_COLS]

# Keep raw backup columns
for c in INPUT_COLS:
    df[f"raw_{c}"] = df[c].fillna("").astype(str)

# Authors
df["authors"] = normalize_author_column(df["authors"])

# Genres
df["genres"] = normalize_genre_column(df["categories"])
df["genres"] = reduce_to_top_genres(df["genres"])

all_genres = (df["genres"].explode().dropna().str.strip())
unique_genres = all_genres.nunique()
# print(f"unique_genres: {unique_genres}")
# print(df["genres"].explode().value_counts())

# normalize titles and descriptions
df["title"] = normalize_text_field(df["title"])
df["description"] = normalize_text_field(df["description"])

In [29]:
# Convert authors list to tuple so it’s hashable
df["authors_tuple"] = df["authors"].apply(tuple)

# Mask of all duplicates (including first occurrence)
duplicate_mask = df.duplicated(subset=["title", "authors_tuple"], keep=False)

# Select and print duplicates
duplicates = df[duplicate_mask]
duplicates

# Optional: drop helper column
# df = df.drop(columns=["authors_tuple"])

,title,description,authors,infolink,categories,raw_title,raw_description,raw_authors,raw_infolink,raw_categories,genres,authors_tuple
206608,"Tribal Talk: Black Theology, Hermeneutics, and...",,[],NaN,NaN,"Tribal Talk: Black Theology, Hermeneutics, and...",,,,,[],()
206641,"Tribal Talk: Black Theology, Hermeneutics, and...",,[],NaN,NaN,"Tribal Talk: Black Theology, Hermeneutics, and...",,,,,[],()


In [ ]:
# Define minimum description length
MIN_DESC_LEN = 10

# Check which books are usable
df["desc_len"] = df["description"].str.len()
df["has_desc"] = df["desc_len"] >= MIN_DESC_LEN
df["has_title"] = df["title"].notna() & df["title"].str.strip() != ""
df["has_author"] = df["authors"].notna() & df["authors"].apply(lambda lst: len(lst) > 0)
df["has_genre"] = df["genres"].apply(lambda lst: len(lst) > 0)

# Boolean mask for usable books
df["is_usable"] = df["has_title"] & df["has_author"] & df["has_desc"] & df["has_genre"]

# Summary counts
usable_counts = df["is_usable"].value_counts()
print(f"Counts of usable books: {usable_counts}")

# Percentage of usable books
usable_percent = df["is_usable"].mean() * 100
print(f"Percentage of usable books: {usable_percent:.2f}%")

# Inspect unusable reasons
unusable_reasons = df.loc[~df["is_usable"], ["title", "authors", "desc_len", "genres", "has_title", "has_author", "has_desc", "has_genre"]]
unusable_reasons.head(10)

is_usable
True     136001
False     76403
Name: count, dtype: int64
Percentage of usable books: 64.03%


,title,authors,desc_len,genres,has_title,has_author,has_desc,has_genre
0,Its Only Art If Its Well Hung!,[Julie Strain],0,[comics and graphic novels],True,True,False,True
4,"Nation Dance: Religion, Identity and Cultural ...",[Edward Long],0,[],True,True,False,False
6,The Overbury affair (Avon),[Miriam Allen De Ford],0,[],True,True,False,False
7,A Walk in the Woods: a Play in Two Acts,[Lee Blessing],0,[],True,True,False,False
25,Homer or Moses?: Early Christian Interpretatio...,[Arthur J. Droge],0,[religion],True,True,False,True
27,"Thomas Jefferson, (American statesmen)",[John T. Morse],0,[],True,True,False,False
29,Dealing With Disappointment: Helping Kids Cope...,[John Adrian],1179,[],True,True,True,False
32,"Little One, Maid of Israel",[E. B. Harvey],0,[other],True,True,False,True
34,Treat yourself to life,[Raymond Charles Barker],0,[other],True,True,False,True
35,Mini-mysteries,[],0,[],True,False,False,False


### `part 3`

In [3]:
df = pd.read_csv("../data/cleaned_books_data.csv")

In [4]:
print(df.shape)
print(list(df.columns))
df.info()

(136001, 5)
['title', 'authors', 'description', 'genres', 'infolink']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136001 entries, 0 to 136000
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        136001 non-null  object
 1   authors      136001 non-null  object
 2   description  136001 non-null  object
 3   genres       136001 non-null  object
 4   infolink     136001 non-null  object
dtypes: object(5)
memory usage: 5.2+ MB


In [5]:
df.isna().sum()

title          0
authors        0
description    0
genres         0
infolink       0
dtype: int64

In [6]:
df.head()

,title,authors,description,genres,infolink
0,Dr. Seuss: American Icon,['Philip Nel'],Philip Nel takes a fascinating look into the k...,['biography and autobiography'],http://books.google.nl/books?id=IjvHQsCn_pgC&d...
1,Wonderful Worship in Smaller Churches,['David R. Ray'],This resource includes twelve principles in un...,['religion'],http://books.google.nl/books?id=2tsDAAAACAAJ&d...
2,Whispers of the Wicked Saints,['Veronica Haddon'],Julia Thomas finds her life spinning out of co...,['fiction'],http://books.google.nl/books?id=aRSIgJlq6JwC&d...
3,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],In The Church of Christ: A Biblical Ecclesiolo...,['religion'],http://books.google.nl/books?id=kVqRaiPlx88C&d...
4,Saint Hyacinth of Poland,['Mary Fabyan Windeatt'],The story for children 10 and up of St. Hyacin...,['biography and autobiography'],http://books.google.nl/books?id=lmLqAAAACAAJ&d...


In [ ]:
type(df.loc[1]['genres'])

str

In [39]:
import ast

def parse_genre_string(genre_str):
    try:
        parsed = ast.literal_eval(genre_str)
        if isinstance(parsed, list):
            return parsed
        return []
    except Exception:
        return []

# Filter rows where at least one genre contains "and"
rows_with_and = df[df["genres"].apply(lambda s: any("and" in g.lower() for g in parse_genre_string(s)))]

# Show results
rows_with_and[["genres"]].head(10)

,genres
0,['biography and autobiography']
4,['biography and autobiography']
9,['technology and engineering']
14,['health and fitness']
18,['sports and recreation']
19,"['body, mind and spirit']"
22,['sports and recreation']
23,['biography and autobiography']
24,['biography and autobiography']
26,['crafts and hobbies']
